<a href="https://colab.research.google.com/github/dD2405/Intro_to_TensorFlow/blob/master/Classifying_images_of_clothing(MNIST).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>